### Conll to note format for sanskrit

In [4]:
# # Rename the files to gold and silver
# import argparse
# import os
# path = './data/train_20k/'
# count = 0
# for file in os.listdir(path):
#         count= count + 1
#         dst ="Silver_" + str(count) + ".csv"
#         src =file
#         os.rename(path+src,path+dst) 

In [8]:
ma_set = set()
word = None
with open('../data/multilingual_word_embeddings/pos_embedding_1_hot.475', 'r', encoding='utf-8') as f:
    # skip first line
    for i, line in enumerate(f):
        if i == 0:
            continue
        word, vec = line.split('@', 1)
        ma_set.add(word)
# ma_set

In [17]:
import re
def clean_ma(ma):
    ma = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []').replace(' ac','').replace(' ps','').replace('sgpl','sg').replace('sgdu','sg')
    ma = ma.replace('i.','inst.').replace('.','').replace(' ','')
    return ma
def IND(ma):
    indeclinable = ['ind','prep','interj','prep','conj','part','abs','ca abs']
    for i in indeclinable:
        if i in ma:
            ma = 'indecl'
    return ma
s = set()
for ma in ma_set:
    s.add(clean_ma(ma))
len(s)

244

In [97]:
import argparse
import os
path = '../data/sanskrit_treebank/'
languages_for_low_resource = ['san']
languages = sorted(list(set(languages_for_low_resource)))
#
splits = ['train','dev','test']
lng_to_files = dict((language, {}) for language in languages)
for language, d in lng_to_files.items():
    for split in splits:
        d[split] = []
    lng_to_files[language] = d
sub_folders = os.listdir(path)
for sub_folder in sub_folders:
#     if sub_folder == 'train':
#         continue
    folder = os.path.join(path, sub_folder)
    files = sorted(os.listdir(folder))
    for file in files:
        full_path = os.path.join(folder, file)
        lng_to_files[language][sub_folder].append(full_path)
###################################
# # # # Add silver in train:
# counter = 0
# amount = 2000
# for f in os.listdir('../data/train_20k/'):
#     full_path = os.path.join('../data/train_20k/', f)
#     lng_to_files[language]['train'].append(full_path)
#     counter += 1
# #     if counter >= amount:
# #         break
    
word_set = set()
for language, split_dict in lng_to_files.items():
    posi =set()
    for split, files in split_dict.items():
        sentences = []
        num_sentences = 0
        for file in sorted(files):
#             if split =='train':
#                 print(file)
            num_sentences += 1
            flag = 0
            with open(file, 'r') as file:
                for line in file:
                    new_line = []
                    line = line.strip()
#                     print(line)
                    if flag == 0:
                        flag = 1
                        continue
                    tokens = line.split(',')
                    id = str(int(tokens[0]) + 1)
                    word = tokens[1]
                    word_set.add(word)
                    ############################
                    # To add rule features id of rule is given
                    name = file.name.replace('../data/sanskrit_treebank/','').replace('.csv','')\
                    .replace('../data/train_20k/','').replace('train/','').replace('dev/','').replace('test/','')
                    lemma = tokens[2]
                    pos = tokens[3]
#                     tag.add(pos)
                    if tokens[6]:
                        arc_tag = tokens[5]
                        head = str(int(tokens[6]) + 1)
                    else:
                        arc_tag = 'root'
                        head = '0'
                    new_line = [id, word ,get_modified_coarse(pos),pos, head, arc_tag,get_case(pos)]
                    posi.add(pos)
#                     print(new_line)
                    sentences.append(new_line)
            sentences.append([])   
#         ud_pos_ner_dp_
        print('Language: %s Split: %s Num. Sentences: %s ' % (language, split, num_sentences))
        if not os.path.exists('data'):
            os.makedirs('data')
#         ud_pos_ner_dp_      Multitask_case_predict_
        write_data_path = '../../../DCST_scratch/data/Multitask_case_predict_' + split + '_' + language
        print('creating %s' % write_data_path)
        with open(write_data_path, 'w') as f:
            for line in sentences:
                 f.write('\t'.join(line) + '\n')
               
              

Language: san Split: train Num. Sentences: 1495 
creating ../../../DCST_scratch/data/Multitask_case_predict_train_san
Language: san Split: dev Num. Sentences: 1000 
creating ../../../DCST_scratch/data/Multitask_case_predict_dev_san
Language: san Split: test Num. Sentences: 1000 
creating ../../../DCST_scratch/data/Multitask_case_predict_test_san


### Extract Coarse tags from 20k data
Use this dictionary

In [90]:
import pandas as pd
paths= ['../data/train_20k/']
ma_20=set()
for path in paths:
    for file in os.listdir(path):
        df = pd.read_csv(path+file,sep=',')
        for i in range(0,len(df)):
                ma_20.add(df.iloc[i,3])
                
def get_modified_coarse_for_20k(ma,coarse_dict_20k):
    ma = ma.replace('sgpl','sg').replace('sgdu','sg')
    for key in coarse_dict_20k.keys():
        if ma in coarse_dict_20k[key]:
            return key

with open('../../L2S/files/coarse_to_ma_dict.json', 'r') as fh:
        coarse_dict = json.load(fh)
coarse_dict_20k = dict()
for key in coarse_dict:
    coarse_dict_20k[key] = []
    for k in coarse_dict[key]:
        coarse_dict_20k[key].append(clean_ma(k))
for x in ['g. du. *.','i. du. *.','abl. pl. *.','i. du. f.']:
    coarse_dict_20k['Noun'].append(clean_ma(x))

In [19]:
def get_case_from_244(ma):
    indeclinable = ['ind','prep','interj','prep','conj','part','indecl']
    case_list = ['nom','voc','acc','inst','dat','abl','loc','i','g']
    gender_list = ['n','f','m','*']
    person_list = ['1','2','3']
    no_list = ['du','sg','pl']
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    # Remove active passive
    case=''
    if ma == 'comp':
        case ='comp'
    for tag in indeclinable:
        if tag in ma:
            case= "IND"

    # Get case
    if case =='':
        for tag in case_list:
            if tag in ma:
                if tag == 'i':
                    if tag+'sg' in ma or tag+'du' in ma or tag+'pl' in ma:
                        case = 'i'
                elif tag == 'g':
                    if tag+'sg' in ma or tag+'du' in ma or tag+'pl' in ma:
                        case = 'g'
                else:
                    case = tag
    if case == '':
        if 'adv' in ma:
            case = 'adv'
    
    if case =='':
        for tag in no_list:
            if tag in ma:
                case = 'FV'
    if case=='':
        for tag in no_list:
            if tag in ma:
                case = 'FV'
    if case=='':
         for tag in person_list:
            if tag in ma:
                case = 'FV'
    if case=='':
        case='IV'
        
    return case
s = []
for m in ma_set:
    s.append([m,get_case_from_244(m)])

### Build 1-hot for POS embedding:

In [19]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
# data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
data = list(tag)
values = array(data)
# print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
print('Dimension is',len(onehot_encoded[0]))
# # invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

Dimension is 475


/home/jivnesh/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [35]:
import json
indeclinable = ['ind','prep','interj','prep','conj','part','abs','ca abs']
case_list = ['nom','voc','acc','i','inst','dat','abl','g','loc']
gender_list = ['n','f','m','*']
person_list = ['1','2','3']
no_list = ['du','sg','pl']
pops = [' ac',' ps']
def get_modified_coarse(ma):
    ma = ma.replace('sgpl','sg').replace('sgdu','sg')
    with open('../../L2S/files/coarse_to_ma_dict.json', 'r') as fh:
        coarse_dict = json.load(fh)
    for key in coarse_dict.keys():
        if ma in coarse_dict[key]:
            return key
get_modified_coarse('pp. . sg. ')  

'FV'

In [47]:
def get_case(ma):
        indeclinable = ['ind','prep','interj','prep','conj','part']
        case_list = ['nom','voc','acc','i','inst','dat','abl','g','loc']
        gender_list = ['n','f','m','*']
        person_list = ['1','2','3']
        no_list = ['du','sg','pl']
        pops = [' ac',' ps']
        ma=ma.replace('sgpl','sg').replace('sgdu','sg')
        temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
        temp = temp.split('.')
        if temp[-1] == '':
            temp.pop(-1)
        # Remove active passive
        case=''
        no=''
        person=''
        gender=''
        tense=''
        coarse=''
        for a,b in enumerate(temp):
            if b in pops:
                temp.pop(a)
        # Get gender
        for a,b in enumerate(temp):
            if b.strip() in gender_list:
                gender = b.strip()
                temp.pop(a)
        # Get case
        for a,b in enumerate(temp):
            if b.strip() in case_list:
                case = b.strip()
                temp.pop(a)
        if case!= '':
            coarse ='Noun'
        # Get person
        for a,b in enumerate(temp):
            if b.strip() in person_list:
                person = b.strip()
                temp.pop(a)
        # Get no
        for a,b in enumerate(temp):
            if b.strip() in no_list:
                no = b.strip()
                temp.pop(a)
        # Get Tense
        for b in temp:
            tense=tense+ ' '+b.strip()
        tense=tense.strip()
        
#         print(tense)
        if tense == 'adv':
            coarse = 'adv'
        for ind in indeclinable:
            if tense == ind:
                coarse = 'Ind'
        if tense == 'abs' or tense == 'ca abs':
            coarse = 'IV'
        if tense!='' and coarse=='':
            if person !='' or no!='':
                coarse= 'FV'
            else:
                coarse = 'IV'
        if case == 'i':
            return 'inst'
        
        if case !='':
            return case
        else:
            return coarse
s = []
for m in ma_set:
    s.append([m,get_case(m)])
sorted(s)

[['abl. du. f.', 'abl'],
 ['abl. du. m.', 'abl'],
 ['abl. du. n.', 'abl'],
 ['abl. pl. f.', 'abl'],
 ['abl. pl. m.', 'abl'],
 ['abl. pl. n.', 'abl'],
 ['abl. sg. *', 'abl'],
 ['abl. sg. f.', 'abl'],
 ['abl. sg. m', 'abl'],
 ['abl. sg. m.', 'abl'],
 ['abl. sg. n', 'abl'],
 ['abl. sg. n.', 'abl'],
 ['abl.sg.m', 'abl'],
 ['abs.', 'IV'],
 ['acc. du. *', 'acc'],
 ['acc. du. f.', 'acc'],
 ['acc. du. m.', 'acc'],
 ['acc. du. n.', 'acc'],
 ['acc. pl. *', 'acc'],
 ['acc. pl. *.', 'acc'],
 ['acc. pl. f.', 'acc'],
 ['acc. pl. m.', 'acc'],
 ['acc. pl. n.', 'acc'],
 ['acc. sg. *', 'acc'],
 ['acc. sg. *.', 'acc'],
 ['acc. sg. f', 'acc'],
 ['acc. sg. f.', 'acc'],
 ['acc. sg. m.', 'acc'],
 ['acc. sg. n', 'acc'],
 ['acc. sg. n.', 'acc'],
 ['adv.', 'adv'],
 ['aor. [1] ac. du. 3', 'FV'],
 ['aor. [1] ac. pl. 3', 'FV'],
 ['aor. [1] ac. sg. 1', 'FV'],
 ['aor. [1] ac. sg. 3', 'FV'],
 ['aor. [2] ac. sg. 3', 'FV'],
 ['aor. [4] ac. du. 2', 'FV'],
 ['aor. [4] ac. sg. 3', 'FV'],
 ['aor. [5] ac. sg. 2', 'FV'],
 ['

In [6]:
def variation_III(ma):
    ##############################################
    # Description:
    # (1) Active passive is removed
    # (2) Tense [*] is removed
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
    temp = temp.split('.')
    if temp[-1] == '':
        temp.pop(-1)
    # Remove active passive
    case=''
    no=''
    person=''
    gender=''
    tense=''

    for a,b in enumerate(temp):
        if b in pops:
            temp.pop(a)
    # Get gender
    for a,b in enumerate(temp):
        if b.strip() in gender_list:
            gender = b.strip()
            temp.pop(a)
    # Get case
    for a,b in enumerate(temp):
        if b.strip() in case_list:
            case = b.strip()
            temp.pop(a)
#     if case == 'i':
#         case = 'inst'
    # Get person
    for a,b in enumerate(temp):
        if b.strip() in person_list:
            person = b.strip()
            temp.pop(a)
    # Get no
    for a,b in enumerate(temp):
        if b.strip() in no_list:
            no = b.strip()
            temp.pop(a)
    # Get Tense
    for b in temp:
        tense=tense+ ' '+b.strip()
    tense=tense.strip()
    var=''

    if case:
        t = 'case=%s '%case
        var=var+t
    if gender:
        t = 'gender=%s '%gender
        var=var+t
    if tense:
        t = 'tense=%s '%tense
        var=var+t
    if no:
        t = 'no=%s '%no
        var=var+t
    if person:
        t = 'person=%s '%person
        var=var+t
    return var.strip()
variation_III('pp. . sg. ')


'tense=pp no=sg'

In [4]:
import re
def variation_IV(ma):
    ma=ma.replace('sgpl','sg').replace('sgdu','sg')
    temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
    temp = temp.split('.')
    if temp[-1] == '':
        temp.pop(-1)
    # Remove active passive
    case=''
    no=''
    person=''
    gender=''
    tense=''
    coarse=''
    for a,b in enumerate(temp):
        if b in pops:
            temp.pop(a)
    # Get gender
    for a,b in enumerate(temp):
        if b.strip() in gender_list:
            gender = b.strip()
            temp.pop(a)
    # Get case
    for a,b in enumerate(temp):
        if b.strip() in case_list:
            case = b.strip()
            temp.pop(a)
    if case!= '':
        coarse ='Noun'
#     if case == 'i':
#         case = 'inst'
    # Get person
    for a,b in enumerate(temp):
        if b.strip() in person_list:
            person = b.strip()
            temp.pop(a)
    # Get no
    for a,b in enumerate(temp):
        if b.strip() in no_list:
            no = b.strip()
            temp.pop(a)
    # Get Tense

    for b in temp:
        tense=tense+ ' '+b.strip()
    tense=tense.strip()
    if tense == 'adv':
        coarse = 'adv'
        
    for ind in indeclinable:
        if tense == ind:
            coarse = 'Ind'

    if tense!='' and coarse=='':
        if person !='' and no!='':
            coarse= 'FV'
        else:
            coarse = 'IV'
    var=''
    if case:
        t = 'case=%s '%case
        var=var+t
    if gender:
        t = 'gender=%s '%gender
        var=var+t
    if tense:
        t = 'tense=%s '%tense
        var=var+t
    if no:
        t = 'no=%s '%no
        var=var+t
    if person:
        t = 'person=%s '%person
        var=var+t
    coarse = get_modified_coarse(ma)
    if coarse:
        t = 'coarse=%s '%coarse
        var=var+t
    return var.strip();
l = []
# for ma in ma_set:
#     l.append(ma+' = '+variation_IV(ma))
variation_IV('pp. . sg. ')

'tense=pp no=sg coarse=FV'

In [25]:
# onehot_encoded[0]
count =0 
with open("pos_embedding_1_hot.475", "w") as f:   
    f.write(str(len(tag))+' '+str(475)+'\n')
    for word in tag:
        word=word+'@'
        f.write(word) 
        for v in onehot_encoded[count]:
            f.write(' '+str(v))
        f.write('\n')
        count = count + 1
print(count)

475
